In [ ]:
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

from pdflight import in_task, shift_datetime

%matplotlib inline

In [ ]:
with open("store.pkl", "rb") as f:
    stored = pickle.load(f)
    md, fl, th = stored["md"], stored["fl"], stored["th"]
md.competition.unique()

In [ ]:
competition = md.index[
    md.competition == "23rd-fai-european-gliding-championships-bekescsaba-2025"
]
if len(competition):
    md = md.loc[competition]
    fl = fl.loc[competition]
    th = th.loc[competition]
fl

In [ ]:
tt = th.loc[in_task(th, md) & (th.dt.total_seconds() >= 120)]
tt.index

In [ ]:
df = tt.reset_index()
df = df.assign(
    h_enter=fl.loc[pd.MultiIndex.from_frame(df[["flight", "datetime"]])].alt.values
)
df = df.assign(
    v_enter=fl.loc[pd.MultiIndex.from_frame(df[["flight", "datetime"]])].gsp.values
)
df = df.assign(exit=df.datetime + df.duration)
df = df.assign(
    h_exit=fl.loc[pd.MultiIndex.from_frame(df[["flight", "exit"]])].alt.values
)
df = df.assign(
    v_exit=fl.loc[pd.MultiIndex.from_frame(df[["flight", "exit"]])].gsp.values
)
df = df.assign(dh=df.h_exit - df.h_enter)
df = df.assign(vv=df.dh / df.duration.dt.total_seconds())
df = df.assign(stable=df.datetime + pd.Timedelta(seconds=40))
df = df.assign(
    h_stable=fl.loc[pd.MultiIndex.from_frame(df[["flight", "stable"]])].alt.values
)
df = df.assign(ds=df.h_stable - df.h_enter)
df = df.assign(vs=df.ds / 40)
df = df.assign(
    dn=df.vs / ((df.h_exit - df.h_stable) / (df.duration.dt.total_seconds() - 40))
)
df

In [ ]:
cc = md.join(df.loc[df.v_enter < 500].groupby("flight").mean(), how="inner")
cc.sort_values("v_enter", ascending=False)

In [ ]:
sns.regplot(data=cc, x="v_exit", y="points", line_kws={"color": "red"})
stats.linregress(cc.v_exit, cc.points)

In [ ]:
sns.histplot(cc.v_exit)